In [1]:
import tensorflow as tf

In [2]:
# import MNIST dataset from tensorflow with one hot encoding style
from tensorflow.examples.tutorials import mnist
mnist = mnist.input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Hyperparams
# Replace LOG_DIR by your log directory
learning_rate = 0.01
training_iteration = 30 
batch_size = 100
display_step = 2
logdir = LOG_DIR

In [4]:
# Inputs
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None, 10])

In [5]:
# Set model params 
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

with tf.name_scope('Wx_b') as scope:
    model = tf.nn.softmax(tf.matmul(x, W) + b)
    
# Add summary stats for weights and bias 
w_h = tf.histogram_summary("weights", W)
b_h = tf.histogram_summary("biases", b)

In [6]:
with tf.name_scope("cost_function") as scope:
    cost_function = -tf.reduce_sum(y * tf.log(model))
    tf.scalar_summary("cost_function", cost_function)

In [7]:
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [8]:
init = tf.initialize_all_variables()

In [9]:
merged_summary_opertion = tf.merge_all_summaries()

In [10]:
# Launch the graph in a session 
with tf.Session() as session:
    session.run(init)
    
    summary_writer = tf.train.SummaryWriter(
        logdir,
        session.graph
    )
    
    for iteration in range(training_iteration):
        average_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            session.run(optimizer, feed_dict={x: batch_xs, y:batch_ys})
            average_cost += session.run(cost_function, feed_dict={x: batch_xs, y:batch_ys})/total_batch
            summary_stats = session.run(merged_summary_opertion, feed_dict={x: batch_xs, y:batch_ys})
            summary_writer.add_summary(summary_stats, iteration+total_batch+i)
        
        if iteration % display_step == 0:
            print "Iteration: {}, Cost: {}".format(iteration+1, average_cost)
        # Test the model 
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))

    # Accuracy 
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print accuracy.eval({x: mnist.test.images, y: mnist.test.labels}, session=session)

Iteration: 1, Cost: 29.8604708897
Iteration: 3, Cost: 20.9778132664
Iteration: 5, Cost: 20.1741527683
Iteration: 7, Cost: 19.6968417523
Iteration: 9, Cost: 19.3454920821
Iteration: 11, Cost: 19.1248761428
Iteration: 13, Cost: 18.9505310778
Iteration: 15, Cost: 18.7957047827
Iteration: 17, Cost: 18.589945814
Iteration: 19, Cost: 18.5861166902
Iteration: 21, Cost: 18.3694002169
Iteration: 23, Cost: 18.376006201
Iteration: 25, Cost: 18.2881719442
Iteration: 27, Cost: 18.2386381435
Iteration: 29, Cost: 18.1866508657
0.9201


# Useful Tips 

#### Use python [Location of tensorboard.py] --logdir [Location of log directory] to visualise scope data
#### Location of tensorboard.py = Location of tensorflow/tensorboard/tensorboard.py
#### Location of tensorflow can be found out by - pip show tensorflow